In [ ]:
def categorize_ethnicity(ethnicity):
    if 'asian' in ethnicity.lower():
        return 'Asian'
    elif 'black' in ethnicity.lower():
        return 'Black'
    elif 'hispanic' in ethnicity.lower():
        return 'Hispanic'
    elif 'native hawaiian' in ethnicity.lower() or 'pacific islander' in ethnicity.lower():
        return 'Native Hawaiian'
    elif 'white' in ethnicity.lower():
        return 'White'
    else:
        return 'Other'

In [100]:
import pandas as pd

# Load your data (admissions, patients, icd_codes, etc.)
admissions = pd.read_csv("../mimic-iii-clinical-database-1.4/ADMISSIONS.csv") # 58976

patients = pd.read_csv("../mimic-iii-clinical-database-1.4/PATIENTS.csv") #46520
admissions = admissions.merge(patients[['SUBJECT_ID', 'GENDER']], on='SUBJECT_ID')

icustays = pd.read_csv("../mimic-iii-clinical-database-1.4/ICUSTAYS.csv")[["SUBJECT_ID", "LOS"]] #61532
admissions = admissions.merge(icustays, on='SUBJECT_ID')
diagnosis = pd.read_csv("../mimic-iii-clinical-database-1.4/DIAGNOSES_ICD.csv") #651047
procedures = pd.read_csv("../mimic-iii-clinical-database-1.4/PROCEDURES_ICD.csv") #38812

# 1. Age Calculation
patients['DOB'] = pd.to_datetime(patients['DOB'])
admissions['ADMITTIME'] = pd.to_datetime(admissions['ADMITTIME'])
admissions = admissions.merge(patients[['SUBJECT_ID', 'DOB']], on='SUBJECT_ID')
admissions['AGE'] = admissions['ADMITTIME'].dt.year - admissions['DOB'].dt.year

# 2. Ethnicity Categorization
admissions['ETHNICITY'] = admissions['ETHNICITY'].apply(categorize_ethnicity)

# 3. Exclusion of In-Hospital Mortalities
admissions = admissions[admissions['HOSPITAL_EXPIRE_FLAG'] != 1]

# 4. Filtering of Diagnosis and Procedure Codes

diagnosis['ICD9_CODE'] = diagnosis['ICD9_CODE'].astype(str)
procedures['ICD9_CODE'] = procedures['ICD9_CODE'].astype(str)
diagnosis = diagnosis.groupby('ICD9_CODE').filter(lambda x: len(x) > 1)
procedures = procedures.groupby('ICD9_CODE').filter(lambda x: len(x) > 1)

# 5. Exclusion of Elective Admissions
admissions = admissions[admissions['ADMISSION_TYPE'] != 'ELECTIVE']

# 6. Age Range Restriction
admissions = admissions[(admissions['AGE'] >= 18) & (admissions['AGE'] <= 90)]


# 7. Mortality Duration Calculation
admissions['DISCHTIME'] = pd.to_datetime(admissions['DISCHTIME'])
admissions['DEATHTIME'] = pd.to_datetime(admissions['DEATHTIME'])
admissions['MORTALITY_1_YEAR'] = ((admissions['DEATHTIME'] - admissions['DISCHTIME']).dt.days <= 365).astype(int)
admissions['MORTALITY_30_DAY'] = ((admissions['DEATHTIME'] - admissions['DISCHTIME']).dt.days <= 30).astype(int)


# 8. ICD Code Concatenation and Differentiation
diagnosis['ICD_TYPE'] = 'diag_'
procedures['ICD_TYPE'] = 'proc_'
diagnosis['ICD'] = diagnosis['ICD_TYPE'] + diagnosis['ICD9_CODE']
procedures['ICD'] = procedures['ICD_TYPE'] + procedures['ICD9_CODE']
combined_icd = pd.concat([diagnosis, procedures]).groupby('HADM_ID')['ICD'].apply(list)
admissions = admissions.merge(combined_icd, on='HADM_ID', how='left')

# 9. Removal of Records with Empty ICD Codes
combined_icd = combined_icd[combined_icd.str.len() > 0]

# 10. Duplicate Admission Removal
admissions.drop_duplicates(subset='HADM_ID', inplace=True)

# 11. Average Length of Stay Calculation
# Assuming LOS is calculated and present in admissions data

# 12. Feature Selection
features = ['GENDER', 'ETHNICITY', 'LOS', 'AGE', 'HADM_ID']
admissions = admissions[features]

# Merge with combined ICD codes
information_table = admissions.merge(combined_icd, on='HADM_ID', how='inner')

# Assuming ICU Stay Length is calculated and present
# information_table['AVG_ICU_STAY_LENGTH'] = ...

# Final INFORMATION table with 36,870 records
print(f"Number of records in the INFORMATION table: {information_table.shape[0]}")
# Convert 'ETHNICITY' and 'GENDER' columns to one-hot encoding
information_table = pd.concat([information_table, pd.get_dummies(information_table['ETHNICITY'], prefix='ETHNICITY')], axis=1)
information_table = pd.concat([information_table, pd.get_dummies(information_table['GENDER'], prefix='GENDER')], axis=1)

# Drop the original 'ETHNICITY' and 'GENDER' columns
information_table.drop(['ETHNICITY', 'GENDER'], axis=1, inplace=True)

# Now 'data' has one-hot encoded columns for 'ETHNICITY' and 'GENDER'
information_table.to_csv("admit_modified.csv", index=False)


Number of records in the INFORMATION table: 35527
